In [0]:
DATA_FILE = '/datascope/subaru/data/targeting/misc/IS/targets_for_laci.csv'
SKY_FILE = '/datascope/subaru/data/targeting/misc/IS/ra150dec2_sky.feather'
FLUXSTD_FILE = '/datascope/subaru/data/targeting/misc/IS/ra150dec2_fluxstd.feather'
FILLER_FILE = '/datascope/subaru/data/targeting/misc/IS/PS1_point_source_ra150dec2_dobos.csv'
POS = (150.05, 2.2)

# DATA_FILE = '/datascope/subaru/data/targeting/misc/IS/PFS_Murphree.csv'
# SKY_FILE = '/datascope/subaru/data/targeting/misc/IS/ra270dec66_sky.feather'
# FLUXSTD_FILE = '/datascope/subaru/data/targeting/misc/IS/ra270dec66_fluxstd.feather'
# POS = (269.7501824794, 65.99602765373128)

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [0]:
from pfs.ga.targeting.data import Observation
from pfs.ga.targeting.io import ObservationSerializer
from pfs.ga.targeting.util.astro import *

In [0]:
s = ObservationSerializer(
    catalog_name = 'sky',
    column_map = {
                'sky_id': 'targetid',
                'ra': 'RA',
                'dec': 'Dec',
            },
)

sky = s.read(SKY_FILE)

In [0]:
s = ObservationSerializer(
    catalog_name = 'fluxstd',
    column_map = {
        'fluxstd_id': 'targetid',
        'ra': 'RA',
        'dec': 'Dec',
        'parallax_error': 'err_parallax',
        'pmra_error': 'err_pmra',
        'pmdec_error': 'err_pmdec',
    },
    # mask = 'lambda df: df["prob_f_star"] > 0.5',
    bands = {
        b: dict(
            filter = f'filter_{b}',                     # Column storing filter names
            # psf_mag = f'psf_mag_{b}',                 # All None in data file
            # psf_mag_err = f'psf_mag_error_{b}',       # All None in data file
            psf_flux = f'psf_flux_{b}',
            psf_flux_err = f'psf_flux_error_{b}',
        ) for b in 'grizy'
    },
    limits = {
        'ps1_g': [16, 19],
        'ps1_g-ps1_r': [0.25, 0.4],
    }
)

fluxstd = s.read(FLUXSTD_FILE)

In [0]:
s = ObservationSerializer(
    catalog_name = 'filler',
    column_map = {
        'obj_id': 'targetid',
    },
    filters = {
        f'{b}_ps1': dict(
            mag = f'{b}PSFMag',
            mag_err = f'{b}PSFMagErr'
        ) for b in 'grizy'
    }
)

filler = s.read(FILLER_FILE)

In [0]:
s = ObservationSerializer(
    catalog_name = 'is',
    column_map = {
        'obj_id': 'targetid',
        'ra': 'RA',
        'dec': 'Dec',
    },
    filters = {
        'i2_hsc': dict(
            flux = 'i2_hsc',
            flux_err = 'i2_hsc_err'
        )

        # 'g_hsc': dict(
        #     mag = 'g_hsc',
        #     mag_err = 'g_hsc_error'
        # ),
        # 'r2_hsc': dict(
        #     mag = 'r2_hsc',
        #     mag_err = 'r2_hsc_error'
        # ),
        # 'i2_hsc': dict(
        #     mag = 'i2_hsc',
        #     mag_err = 'i2_hsc_error'
        # ),
    }
)
obs = s.read(DATA_FILE)

In [0]:
obs.data

In [0]:
obs.data['RA'].mean(), obs.data['Dec'].mean()

In [0]:
obs.data['exptime'].unique()

In [0]:
# Histogram of exposure times
hist, bins = np.histogram(obs.data['exptime'])

plt.step(0.5 * (bins[1:] + bins[:-1]), hist, where='mid')

In [0]:
 # Magnitude distribution

m = nJy_to_ABmag(obs.data['i2_hsc'])
mask = np.isfinite(m)
hist, bins = np.histogram(m[mask])

plt.step(0.5 * (bins[1:] + bins[:-1]), hist, where='mid')



In [0]:
from pfs.ga.targeting.projection import Pointing, WcsProjection
from pfs.ga.targeting.instrument import *
from pfs.ga.targeting.diagram import *
from astropy.time import Time

In [0]:
pointing = Pointing(*POS, obs_time=Time('2025-03-25T10:00:00'))

In [0]:
wfc = SubaruWFC(pointing)
wcs = WcsProjection(pointing, proj='TAN')
fov = FOV(projection=wcs)

In [0]:
f, ax = plt.subplots(1, 1, figsize=(8, 8), dpi=240, subplot_kw=dict(projection=wcs.wcs))

fov.plot_observation(ax, sky, c='blue', size=0.1)
fov.plot_observation(ax, obs, c='red', size=0.5)
fov.plot_observation(ax, fluxstd, c='green', size=2)
fov.plot_observation(ax, filler, c='yellow', size=1)

# Plot the poiting center
# fov.plot(ax, [field.pos.ra.value], [field.pos.dec.value], fmt='+r', ms=10, native_frame='world')

for pp in [ 
        pointing,

        # Pointing.from_relative_pos(POS, sep=1.15, dir=30, posang=0),
        # Pointing.from_relative_pos(POS, sep=1.15, dir=90, posang=0),
        # Pointing.from_relative_pos(POS, sep=1.15, dir=150, posang=0),
        # Pointing.from_relative_pos(POS, sep=1.15, dir=210, posang=0),
        # Pointing.from_relative_pos(POS, sep=1.15, dir=270, posang=0),
        # Pointing.from_relative_pos(POS, sep=1.15, dir=330, posang=0),
    ]:

    wfc = SubaruWFC(pp)
    pfi = SubaruPFI(wfc, instrument_options={'layout': 'calibration'})

    pfi.plot_focal_plane(ax, fov, corners=True, c='r', ls='-')
    pfi.plot_focal_plane(ax, fov, fill=True, alpha=0.2)

# ax.set_xlim(0, -2)
# ax.set_ylim(-2, 0)

ax.set_xlim(2, -4)
ax.set_ylim(-4, 2)

ax.grid()